In [1]:
#Importing dependencies
import config
import requests
import pandas as pd
import numpy as np
import datetime
from flask import Flask
from sqlalchemy import create_engine
import json

#Avoiding setting with copy warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing Redfin Housing Market State Data
raw_county_df = pd.read_csv("Resources/county_market_tracker.tsv000", sep='\t')
raw_county_df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-04-01,2017-04-30,30,county,5,170,f,"Morgan County, AL",NaN,Alabama,...,-0.069595,NaN,NaN,NaN,0.020833,0.020833,0.001225,"Decatur, AL",19460.0,2022-01-09 14:29:56
1,2014-12-01,2014-12-31,30,county,5,1457,f,"Hennepin County, MN",NaN,Minnesota,...,-0.098136,0.097744,-0.121909,-0.197220,0.172414,0.034483,-0.033469,"Minneapolis, MN",33460.0,2022-01-09 14:29:56
2,2018-02-01,2018-02-28,30,county,5,2319,f,"Tulsa County, OK",NaN,Oklahoma,...,-0.024840,0.206004,-0.026139,0.009313,0.479924,0.068277,-0.042568,"Tulsa, OK",46140.0,2022-01-09 14:29:56
3,2017-08-01,2017-08-31,30,county,5,1448,f,"Crow Wing County, MN",NaN,Minnesota,...,NaN,0.200000,-0.050000,NaN,NaN,NaN,NaN,"Brainerd, MN",14660.0,2022-01-09 14:29:56
4,2019-05-01,2019-05-31,30,county,5,1586,f,"Tate County, MS",NaN,Mississippi,...,0.107895,NaN,NaN,NaN,0.000000,-0.111111,-0.100000,"Memphis, TN",32820.0,2022-01-09 14:29:56


In [3]:
#Printing out column values to figure out what data we are interested in
print(raw_county_df.columns)

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

Property Type

Property type defined by RedFin
- All Residential: All properties defined as single-family, condominium, co-operative, townhouses, and multi-family (2-4 units) homes with a county record.
- Single Family Home (SFH): are homes built on a single lot, with no shared walls. Sometimes there’s a garage, attached or detached.
- Condominium (Condo): Usually a single unit within a larger building or community. Generally come with homeowners’ associations (HOAs), which require the residents to pay monthly or yearly dues.
- Cooperatives (Co-op): Usually a single unit within a larger building or community, but with a different way of holding a title to a shared building. You join a community and everyone in the community owns the building together.
- Townhouse: a hybrid between a condo and a single-family home. They are often multiple floors, with one or two shared walls, and some have a small yard space or rooftop deck. They’re generally larger than a condo, but smaller than a single-family home.
- Multifamily (2-4 units): They are essentially a home that has been turned into two or more units but the units cannot be purchased individually. There is one owner for the whole building.
- Land: Just land, no home of any type for sale.

Columns:

- Home sales: Total number of homes with a sale date during a given time period.
- Total active listings: The total number of listings that were active at any point during a given time period.
- Median sale price: The final home sale price covering all homes with a sale date during a given time period where 50% of the sales were above this price and 50% were below this price.
- Median sale price per square foot: The final home sale price divided by the total square feet of the property (not the lot) covering all homes with a sale date during a given time period where 50% of the sales were above this price per sqft and 50% were below this price per sqft.


In [4]:
#Checking for outliers
raw_county_df['property_type'].value_counts()

All Residential              179003
Single Family Residential    178662
Condo/Co-op                   74594
Multi-Family (2-4 Unit)       70143
Townhouse                     60720
Name: property_type, dtype: int64

In [5]:
# calling only the columns we want to keep for our analysis
county_df = raw_county_df[["period_end","region","parent_metro_region","state","property_type","inventory","homes_sold","median_sale_price","median_ppsf"]]
county_df.head()

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2017-04-30,"Morgan County, AL","Decatur, AL",Alabama,All Residential,439.0,106.0,120500.0,75.845434
1,2014-12-31,"Hennepin County, MN","Minneapolis, MN",Minnesota,Multi-Family (2-4 Unit),133.0,57.0,203000.0,86.633333
2,2018-02-28,"Tulsa County, OK","Tulsa, OK",Oklahoma,All Residential,2665.0,665.0,167500.0,89.765580
3,2017-08-31,"Crow Wing County, MN","Brainerd, MN",Minnesota,Condo/Co-op,5.0,3.0,125000.0,147.076613
4,2019-05-31,"Tate County, MS","Memphis, TN",Mississippi,Single Family Residential,80.0,19.0,165000.0,85.000000


In [6]:
# convert period_end to datetime
county_df["period_end"] =  pd.to_datetime(county_df["period_end"])
county_df

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2017-04-30,"Morgan County, AL","Decatur, AL",Alabama,All Residential,439.0,106.0,120500.0,75.845434
1,2014-12-31,"Hennepin County, MN","Minneapolis, MN",Minnesota,Multi-Family (2-4 Unit),133.0,57.0,203000.0,86.633333
2,2018-02-28,"Tulsa County, OK","Tulsa, OK",Oklahoma,All Residential,2665.0,665.0,167500.0,89.765580
3,2017-08-31,"Crow Wing County, MN","Brainerd, MN",Minnesota,Condo/Co-op,5.0,3.0,125000.0,147.076613
4,2019-05-31,"Tate County, MS","Memphis, TN",Mississippi,Single Family Residential,80.0,19.0,165000.0,85.000000
...,...,...,...,...,...,...,...,...,...
563117,2013-03-31,"Gonzales County, TX",Texas nonmetropolitan area,Texas,All Residential,28.0,2.0,291500.0,83.404488
563118,2013-09-30,"Valley County, ID",Idaho nonmetropolitan area,Idaho,Townhouse,7.0,1.0,221500.0,131.923764
563119,2016-06-30,"Livingston Parish, LA","Baton Rouge, LA",Louisiana,All Residential,602.0,197.0,173990.0,108.063175
563120,2017-09-30,"Gloucester County, VA","Virginia Beach, VA",Virginia,Single Family Residential,79.0,9.0,170000.0,113.502506


In [7]:
# filtering out all data before 2019
county_df = county_df.loc[county_df['period_end'] > '2018-12-31']
county_df

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
4,2019-05-31,"Tate County, MS","Memphis, TN",Mississippi,Single Family Residential,80.0,19.0,165000.0,85.000000
6,2019-12-31,"Madison County, AL","Huntsville, AL",Alabama,Multi-Family (2-4 Unit),8.0,4.0,78500.0,34.880015
11,2021-01-31,"Hood River County, OR","Hood River, OR",Oregon,All Residential,18.0,21.0,650000.0,300.167926
12,2020-10-31,"Lincoln County, CO",Colorado nonmetropolitan area,Colorado,Single Family Residential,9.0,8.0,152450.0,104.869225
14,2021-10-31,"Murray County, OK",Oklahoma nonmetropolitan area,Oklahoma,Single Family Residential,50.0,13.0,185500.0,105.000000
...,...,...,...,...,...,...,...,...,...
563094,2020-10-31,"Hood River County, OR","Hood River, OR",Oregon,Condo/Co-op,1.0,1.0,265000.0,207.517619
563099,2021-01-31,"Columbia County, WA",Washington nonmetropolitan area,Washington,Single Family Residential,10.0,2.0,386000.0,187.768947
563102,2021-02-28,"Lancaster County, PA","Lancaster, PA",Pennsylvania,All Residential,365.0,375.0,239500.0,141.414640
563110,2020-06-30,"Sauk County, WI","Baraboo, WI",Wisconsin,Single Family Residential,264.0,75.0,244900.0,125.218659


In [8]:
# sorting by date and resetting index
county_df = county_df.sort_values(by=['period_end'],ascending=False).reset_index(drop=True)
county_df

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2021-12-31,"Calhoun County, AL","Anniston, AL",Alabama,All Residential,229.0,124.0,165250.0,100.791300
1,2021-12-31,"Tattnall County, GA",Georgia nonmetropolitan area,Georgia,All Residential,6.0,6.0,227450.0,121.116424
2,2021-12-31,"Jefferson County, CO","Denver, CO",Colorado,All Residential,242.0,836.0,570500.0,272.355277
3,2021-12-31,"Monroe County, MI","Monroe, MI",Michigan,Multi-Family (2-4 Unit),13.0,6.0,117850.0,75.228783
4,2021-12-31,"Gila County, AZ","Payson, AZ",Arizona,All Residential,62.0,38.0,425000.0,236.012212
...,...,...,...,...,...,...,...,...,...
184313,2019-01-31,"Walworth County, WI","Whitewater, WI",Wisconsin,Single Family Residential,450.0,70.0,191000.0,123.975410
184314,2019-01-31,"Haywood County, NC","Asheville, NC",North Carolina,All Residential,447.0,52.0,201750.0,132.706230
184315,2019-01-31,"Douglas County, OR","Roseburg, OR",Oregon,Multi-Family (2-4 Unit),6.0,7.0,249000.0,101.461039
184316,2019-01-31,"Buffalo County, WI",Wisconsin nonmetropolitan area,Wisconsin,All Residential,48.0,7.0,128000.0,76.333333


In [9]:
# checking for null values
county_df.isnull().sum()

period_end                0
region                    0
parent_metro_region       0
state                     0
property_type             0
inventory              7657
homes_sold              303
median_sale_price       303
median_ppsf            2196
dtype: int64

In [10]:
# dropping rows without values for homes sold and median sale price
clean_county_df = county_df.dropna(subset = ["homes_sold"])
clean_county_df.head()

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2021-12-31,"Calhoun County, AL","Anniston, AL",Alabama,All Residential,229.0,124.0,165250.0,100.791300
1,2021-12-31,"Tattnall County, GA",Georgia nonmetropolitan area,Georgia,All Residential,6.0,6.0,227450.0,121.116424
2,2021-12-31,"Jefferson County, CO","Denver, CO",Colorado,All Residential,242.0,836.0,570500.0,272.355277
3,2021-12-31,"Monroe County, MI","Monroe, MI",Michigan,Multi-Family (2-4 Unit),13.0,6.0,117850.0,75.228783
4,2021-12-31,"Gila County, AZ","Payson, AZ",Arizona,All Residential,62.0,38.0,425000.0,236.012212


In [11]:
# checking for additional null values
clean_county_df.isnull().sum()

period_end                0
region                    0
parent_metro_region       0
state                     0
property_type             0
inventory              7646
homes_sold                0
median_sale_price         0
median_ppsf            1893
dtype: int64

In [12]:
# dropping rows with null values in the median price per square foot column
clean_county_df = clean_county_df.dropna(subset = ["median_ppsf"])
clean_county_df.head()

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2021-12-31,"Calhoun County, AL","Anniston, AL",Alabama,All Residential,229.0,124.0,165250.0,100.791300
1,2021-12-31,"Tattnall County, GA",Georgia nonmetropolitan area,Georgia,All Residential,6.0,6.0,227450.0,121.116424
2,2021-12-31,"Jefferson County, CO","Denver, CO",Colorado,All Residential,242.0,836.0,570500.0,272.355277
3,2021-12-31,"Monroe County, MI","Monroe, MI",Michigan,Multi-Family (2-4 Unit),13.0,6.0,117850.0,75.228783
4,2021-12-31,"Gila County, AZ","Payson, AZ",Arizona,All Residential,62.0,38.0,425000.0,236.012212


In [13]:
# checking for remaining null values
clean_county_df.isnull().sum()

period_end                0
region                    0
parent_metro_region       0
state                     0
property_type             0
inventory              7444
homes_sold                0
median_sale_price         0
median_ppsf               0
dtype: int64

In [14]:
# not sure if we should drop those rows or keep?

In [15]:
# when doing test queries in sql I noticed that union county and fayette county in OH had higher sale values thatn NY and Nantucket
# checking high sale values to see if there are other values that might contain errors
test_df = clean_county_df.loc[clean_county_df['median_sale_price'] > 5000000].sort_values(by=['median_sale_price'],ascending=False)
test_df
# we need to drop index #89956 and #81782 - those values cannot be correct, right?

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
89956,2020-08-31,"Union County, OH","Columbus, OH",Ohio,Multi-Family (2-4 Unit),NaN,1.0,999999999.0,1.000000e+09
81782,2020-09-30,"Fayette County, OH","Washington Court House, OH",Ohio,Condo/Co-op,1.0,2.0,500099999.5,4.055778e+04
157268,2019-06-30,"New York County, NY","New York, NY",New York,Townhouse,19.0,4.0,20650000.0,2.560714e+03
84410,2020-09-30,"New York County, NY","New York, NY",New York,Townhouse,14.0,1.0,19920000.0,2.735888e+03
95590,2020-07-31,"New York County, NY","New York, NY",New York,Single Family Residential,91.0,4.0,17250000.0,1.363636e+03
...,...,...,...,...,...,...,...,...,...
10691,2021-11-30,"New York County, NY","New York, NY",New York,Multi-Family (2-4 Unit),73.0,11.0,5150000.0,8.152571e+02
134986,2019-11-30,"New York County, NY","New York, NY",New York,Single Family Residential,95.0,5.0,5125000.0,2.450000e+03
103071,2020-05-31,"New York County, NY","New York, NY",New York,Multi-Family (2-4 Unit),96.0,4.0,5100000.0,1.220947e+03
170646,2019-03-31,"New York County, NY","New York, NY",New York,Multi-Family (2-4 Unit),132.0,6.0,5075000.0,1.502503e+03


In [16]:
clean_county_df = clean_county_df.drop([89956,81782])
clean_county_df

,period_end,region,parent_metro_region,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2021-12-31,"Calhoun County, AL","Anniston, AL",Alabama,All Residential,229.0,124.0,165250.0,100.791300
1,2021-12-31,"Tattnall County, GA",Georgia nonmetropolitan area,Georgia,All Residential,6.0,6.0,227450.0,121.116424
2,2021-12-31,"Jefferson County, CO","Denver, CO",Colorado,All Residential,242.0,836.0,570500.0,272.355277
3,2021-12-31,"Monroe County, MI","Monroe, MI",Michigan,Multi-Family (2-4 Unit),13.0,6.0,117850.0,75.228783
4,2021-12-31,"Gila County, AZ","Payson, AZ",Arizona,All Residential,62.0,38.0,425000.0,236.012212
...,...,...,...,...,...,...,...,...,...
184313,2019-01-31,"Walworth County, WI","Whitewater, WI",Wisconsin,Single Family Residential,450.0,70.0,191000.0,123.975410
184314,2019-01-31,"Haywood County, NC","Asheville, NC",North Carolina,All Residential,447.0,52.0,201750.0,132.706230
184315,2019-01-31,"Douglas County, OR","Roseburg, OR",Oregon,Multi-Family (2-4 Unit),6.0,7.0,249000.0,101.461039
184316,2019-01-31,"Buffalo County, WI",Wisconsin nonmetropolitan area,Wisconsin,All Residential,48.0,7.0,128000.0,76.333333


In [17]:
clean_county_df["county"] = clean_county_df["region"].str[:-2]
clean_county_df["county"] = clean_county_df["county"] + clean_county_df["state"]
clean_county_df["year"] = pd.DatetimeIndex(clean_county_df["period_end"]).year

clean_county_df = clean_county_df[["period_end","year","county","state","property_type","inventory","homes_sold","median_sale_price","median_ppsf"]]
clean_county_df

,period_end,year,county,state,property_type,inventory,homes_sold,median_sale_price,median_ppsf
0,2021-12-31,2021,"Calhoun County, Alabama",Alabama,All Residential,229.0,124.0,165250.0,100.791300
1,2021-12-31,2021,"Tattnall County, Georgia",Georgia,All Residential,6.0,6.0,227450.0,121.116424
2,2021-12-31,2021,"Jefferson County, Colorado",Colorado,All Residential,242.0,836.0,570500.0,272.355277
3,2021-12-31,2021,"Monroe County, Michigan",Michigan,Multi-Family (2-4 Unit),13.0,6.0,117850.0,75.228783
4,2021-12-31,2021,"Gila County, Arizona",Arizona,All Residential,62.0,38.0,425000.0,236.012212
...,...,...,...,...,...,...,...,...,...
184313,2019-01-31,2019,"Walworth County, Wisconsin",Wisconsin,Single Family Residential,450.0,70.0,191000.0,123.975410
184314,2019-01-31,2019,"Haywood County, North Carolina",North Carolina,All Residential,447.0,52.0,201750.0,132.706230
184315,2019-01-31,2019,"Douglas County, Oregon",Oregon,Multi-Family (2-4 Unit),6.0,7.0,249000.0,101.461039
184316,2019-01-31,2019,"Buffalo County, Wisconsin",Wisconsin,All Residential,48.0,7.0,128000.0,76.333333


In [32]:
grouped_df = clean_county_df.groupby(["state","county","year","property_type"]).agg({'inventory':'mean','homes_sold':'sum','median_sale_price':'mean', 'median_ppsf': 'mean'})
grouped_df.head(50)

inventory  \
state   county                   year property_type                           
Alabama Bibb County, Alabama     2019 All Residential             22.818182   
                                      Single Family Residential   22.818182   
                                 2020 All Residential             14.916667   
                                      Single Family Residential   14.916667   
                                 2021 All Residential             16.083333   
                                      Single Family Residential   16.083333   
        Blount County, Alabama   2019 All Residential            182.916667   
                                      Single Family Residential  182.666667   
                                 2020 All Residential            150.833333   
                                      Condo/Co-op                       NaN   
                                      Multi-Family (2-4 Unit)           NaN   
                                      Single Family Residential  150.333333   
                                 2021 All Residential            116.833333   
                                      Single Family Residential  116.750000   
        Calhoun County, Alabama  2019 All Residential            432.666667   
                                      Condo/Co-op                  1.000000   
                                      Multi-Family (2-4 Unit)      6.000000   
                                      Single Family Residential  420.083333   
                                      Townhouse                    6.777778   
                                 2020 All Residential            369.083333   
                                      Condo/Co-op                       NaN   
                                      Multi-Family (2-4 Unit)      1.666667   
                                      Single Family Residential  363.333333   
                                      Townhouse                    3.818182   
                                 2021 All Residential            262.666667   
                                      Condo/Co-op                       NaN   
                                      Multi-Family (2-4 Unit)      2.428571   
                                      Single Family Residential  258.166667   
                                      Townhouse                    2.600000   
        Cherokee County, Alabama 2019 All Residential            141.583333   
                                      Condo/Co-op                  2.250000   
                                      Single Family Residential  139.750000   
                                      Townhouse                         NaN   
                                 2020 All Residential            102.916667   
                                      Condo/Co-op                  1.666667   
                                      Single Family Residential  101.083333   
                                      Townhouse                         NaN   
                                 2021 All Residential             67.416667   
                                      Condo/Co-op                  1.750000   
                                      Single Family Residential   65.250000   
        Chilton County, Alabama  2020 All Residential             18.000000   
                                      Single Family Residential   17.000000   
                                 2021 All Residential             76.916667   
                                      Single Family Residential   76.750000   
                                      Townhouse                         NaN   
        Clay County, Alabama     2019 All Residential             24.000000   
                                      Single Family Residential   24.000000   
                                 2020 All Residential             19.666667   
                                      Single Family Residential   19.666667   
                                 2021

# Census Bureau Data

In [19]:
#Importing 2020 - 2021 census bureau data
cb2021_df = pd.read_csv("Resources/co-est2021-alldata.csv", encoding='latin-1')

In [20]:
cb2021_df["county"] = ""
for i in range(len(cb2021_df["CTYNAME"])):
    if cb2021_df["CTYNAME"][i] != cb2021_df["STNAME"][i]:
        cb2021_df["county"][i] = cb2021_df["CTYNAME"][i] + ", " + cb2021_df["STNAME"][i]
    else:
        cb2021_df["county"][i] = "-"

cb2021_df = cb2021_df[["county", "STNAME", "POPESTIMATE2020","POPESTIMATE2021"]]
cb2021_df = cb2021_df.rename(columns={"STNAME": "state", "ESTIMATESBASE2020":"estimates_base_2020","POPESTIMATE2020":"pop_estimate_2020", "POPESTIMATE2021":"pop_estimate_2021"})
cb2021_df

,county,state,pop_estimate_2020,pop_estimate_2021
0,-,Alabama,5024803,5039877
1,"Autauga County, Alabama",Alabama,58877,59095
2,"Baldwin County, Alabama",Alabama,233140,239294
3,"Barbour County, Alabama",Alabama,25180,24964
4,"Bibb County, Alabama",Alabama,22223,22477
...,...,...,...,...
3189,"Sweetwater County, Wyoming",Wyoming,42158,41614
3190,"Teton County, Wyoming",Wyoming,23347,23575
3191,"Uinta County, Wyoming",Wyoming,20441,20635
3192,"Washakie County, Wyoming",Wyoming,7658,7705


In [21]:
#Importing 2019 census bureau data
cb2019_df = pd.read_csv("Resources/co-est2019-alldata.csv", encoding='latin-1')

In [22]:
cb2019_df["county"] = ""
for i in range(len(cb2019_df["CTYNAME"])):
    if cb2019_df["CTYNAME"][i] != cb2019_df["STNAME"][i]:
        cb2019_df["county"][i] = cb2019_df["CTYNAME"][i] + ", " + cb2019_df["STNAME"][i]
    else:
        cb2019_df["county"][i] = "-"

cb2019_df = cb2019_df[["county", "STNAME", "POPESTIMATE2019"]]
cb2019_df = cb2019_df.rename(columns={"STNAME": "state", "POPESTIMATE2019":"pop_estimate_2019"})
cb2019_df

,county,state,pop_estimate_2019
0,-,Alabama,4903185
1,"Autauga County, Alabama",Alabama,55869
2,"Baldwin County, Alabama",Alabama,223234
3,"Barbour County, Alabama",Alabama,24686
4,"Bibb County, Alabama",Alabama,22394
...,...,...,...
3188,"Sweetwater County, Wyoming",Wyoming,42343
3189,"Teton County, Wyoming",Wyoming,23464
3190,"Uinta County, Wyoming",Wyoming,20226
3191,"Washakie County, Wyoming",Wyoming,7805


In [23]:
census_df = cb2019_df.merge(cb2021_df, on=["county","state"])
census_df

,county,state,pop_estimate_2019,pop_estimate_2020,pop_estimate_2021
0,-,Alabama,4903185,5024803,5039877
1,"Autauga County, Alabama",Alabama,55869,58877,59095
2,"Baldwin County, Alabama",Alabama,223234,233140,239294
3,"Barbour County, Alabama",Alabama,24686,25180,24964
4,"Bibb County, Alabama",Alabama,22394,22223,22477
...,...,...,...,...,...
3189,"Sweetwater County, Wyoming",Wyoming,42343,42158,41614
3190,"Teton County, Wyoming",Wyoming,23464,23347,23575
3191,"Uinta County, Wyoming",Wyoming,20226,20441,20635
3192,"Washakie County, Wyoming",Wyoming,7805,7658,7705


In [24]:
clean_census_df = census_df.loc[census_df['county'] != "-"]
clean_census_df

,county,state,pop_estimate_2019,pop_estimate_2020,pop_estimate_2021
1,"Autauga County, Alabama",Alabama,55869,58877,59095
2,"Baldwin County, Alabama",Alabama,223234,233140,239294
3,"Barbour County, Alabama",Alabama,24686,25180,24964
4,"Bibb County, Alabama",Alabama,22394,22223,22477
5,"Blount County, Alabama",Alabama,57826,59081,59041
...,...,...,...,...,...
3189,"Sweetwater County, Wyoming",Wyoming,42343,42158,41614
3190,"Teton County, Wyoming",Wyoming,23464,23347,23575
3191,"Uinta County, Wyoming",Wyoming,20226,20441,20635
3192,"Washakie County, Wyoming",Wyoming,7805,7658,7705


# Load into pgAdmin

In [25]:
# !Must create housing_db database in pgAdmin before running this! 
# schema.sql file has code to create table named county_data

# Structure the database url from the config.py file and create the database engine
url = f'{config.user}:{config.password}@{config.hostname}:{config.port}/housing_db'
engine = create_engine(f'postgresql://{url}', pool_pre_ping=True)

In [27]:
# After running schema.sql in your housing_db, check that the table exists
engine.table_names()

['county_data', 'pop_data']

In [28]:
# Load the housing DataFrame to the database
clean_county_df.to_sql(name='county_data', con=engine, if_exists='append', index=False)

In [29]:
# Load the population estimates DataFrame to the database
clean_census_df.to_sql(name='pop_data', con=engine, if_exists='append', index=False)